# Download ImageNet 1000 images

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

http://image-net.org/challenges/LSVRC/2014/browse-synsets

In [3]:
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions

In [4]:
winds = decode_predictions(np.expand_dims(np.zeros(1000), 0), top=1000)

In [5]:
import os
import requests
import shutil
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool

In [ ]:
data_path = '/data/datasets/imagenet2012/'
image_formats = ("image/png", "image/jpeg", "image/jpg")

In [16]:
def extension_mapper(ct):
    return ct.split('/')[-1]    

def download_image(ind, l, wind_path):
#     l = list_of_images[ind]
    try:
        r = requests.get(l, timeout=8)
        status_code = r.status_code
        if status_code == 200 and r.headers['Content-Type'] in image_formats:
            ext = extension_mapper(r.headers['Content-Type'])
            fname = '{}/{}.{}'.format(wind_path, ind, ext)

            response = {'ind':ind,
                        'url': l, 
                        'fname': fname,
                        'status_code': status_code}
            img_data = r.content
            with open(fname, 'wb') as f:
#                 r.raw.decode_content = True
#                 shutil.copyfileobj(r.raw, f)
                f.write(img_data)
        else:
            response = {'ind':ind,
            'url': l, 
            'fname': None,
            'status_code': status_code}
    except:
        response = {'ind':ind,
                    'url': l, 
                    'fname': None,
                    'status_code': 'err/to'}
    return response

In [ ]:
# wind = 'n02391049'
pool = Pool(23)
for wind in tqdm(winds[0]):
    wind = wind[0]
#     print(wind)
    wind_path = '{}/{}'.format(data_path, wind)
    os.makedirs(wind_path, exist_ok=True)
    resp = requests.get('http://www.image-net.org/api/text/imagenet.synset.geturls?wnid={wind}'.format(wind=wind))
    list_of_images = [x.replace('\n', '') for x in resp.text.split('\r')]
#     list_of_images = list_of_images[:20]
    inds = range(len(list_of_images))
    p_aux = [wind_path for x in inds]
    results = pool.starmap(download_image, zip(inds, list_of_images, p_aux))

    pd_results = pd.DataFrame(results)
    restuls_file = '{}/results.json'.format(wind_path)
    pd_results.to_json(restuls_file)

pool.close()

  8%|▊         | 81/1000 [2:33:27<38:44:08, 151.74s/it]

### Bradley images

PubFig: Public Figures Face Database

https://www.cs.columbia.edu/CAVE/databases/pubfig/download/

In [11]:
url = 'https://www.cs.columbia.edu/CAVE//databases/pubfig/download/eval_urls.txt'
r = requests.get(url)

In [23]:
list_of_c = [x.split('\t') for x in r.text.split('\n')]

In [27]:
list_of_c[1]

['#', 'person', 'imagenum', 'url', 'rect', 'md5sum']

In [28]:
list_of_c[2]

['Aaron Eckhart',
 '1',
 'http://farm1.static.flickr.com/119/288329997_19ebf1d7b3_o.jpg',
 '248,92,338,182',
 'a980a9e21c90ff62e57345fad53a56c8']

In [31]:
df = pd.DataFrame(list_of_c[2:-1], columns = list_of_c[1][1:] )
df_brad = df[df['person'] == 'Brad Pitt'] 

download images

In [37]:
wind = 'brad'

# wind = 'n02391049'
pool = Pool(23)

#     print(wind)
wind_path = '{}/{}'.format(data_path, wind)
os.makedirs(wind_path, exist_ok=True)

#     list_of_images = list_of_images[:20]
list_of_images = df_brad['url'].values
inds = range(len(list_of_images))
p_aux = [wind_path for x in inds]
results = pool.starmap(download_image, zip(inds, list_of_images, p_aux))

pd_results = pd.DataFrame(results)
restuls_file = '{}/results.json'.format(wind_path)
pd_results.to_json(restuls_file)

pool.close()

In [46]:
wind_path

'/data/datasets/imagenet2012//teddy'

In [49]:
pd.read_json('/data/datasets/imagenet2012/brad/curated_results.json')

,ind,url,fname,status_code
0,0,http://farm1.static.flickr.com/43/100299048_85...,/data/datasets/imagenet2012//brad/0.jpeg,200
1,1,http://farm1.static.flickr.com/39/100299580_df...,/data/datasets/imagenet2012//brad/1.jpeg,200
2,2,http://farm1.static.flickr.com/8/10090770_9569...,/data/datasets/imagenet2012//brad/2.jpeg,200
3,3,http://farm1.static.flickr.com/19/106208948_14...,/data/datasets/imagenet2012//brad/3.jpeg,200
4,4,http://farm1.static.flickr.com/40/106209220_f5...,/data/datasets/imagenet2012//brad/4.jpeg,200
...,...,...,...,...
1050,1050,http://i7.tinypic.com/8gegj6a.jpg,/data/datasets/imagenet2012//brad/1050.jpeg,200
1051,1051,http://i8.tinypic.com/7wza2x4.jpg,/data/datasets/imagenet2012//brad/1051.jpeg,200
1052,1052,http://i9.tinypic.com/449aohx.jpg,/data/datasets/imagenet2012//brad/1052.jpeg,200
1053,1053,http://i9.tinypic.com/67rmqmf.jpg,/data/datasets/imagenet2012//brad/1053.jpeg,200


### teddy bear (imagenet links are broken)

https://github.com/naveenchanakya/bear-classifier/blob/master/teddybear.txt

In [10]:
url = 'https://raw.githubusercontent.com/naveenchanakya/bear-classifier/master/teddybear.txt'
r = requests.get(url)

In [11]:
list_of_c = [x for x in r.text.split('\n')]

In [18]:
data_path

'/data/datasets/imagenet2012/'

In [17]:
wind = 'teddy'

# wind = 'n02391049'
pool = Pool(23)

#     print(wind)
wind_path = '{}/{}'.format(data_path, wind)
os.makedirs(wind_path, exist_ok=True)

#     list_of_images = list_of_images[:20]
list_of_images = list_of_c
inds = range(len(list_of_images))
p_aux = [wind_path for x in inds]
results = pool.starmap(download_image, zip(inds, list_of_images, p_aux))

pd_results = pd.DataFrame(results)
restuls_file = '{}/results.json'.format(wind_path)
pd_results.to_json(restuls_file)

pool.close()

### generate image codes and categories

In [42]:
from tensorflow.keras.applications.resnet50 import decode_predictions
preds_codes = decode_predictions(np.expand_dims(np.arange(1000)[::-1], axis=0), top=1000)[0]
# preds_codes = decode_predictions(np.expand_dims(np.zeros(1000), axis=0), top=1000)[0]
cats = []
for i, (code, desc, _) in enumerate(preds_codes):
    cats.append([i, code, desc])

In [43]:
cats.append([i+1, 'brad', 'W Bradley Pitt, aka Brad'])

In [44]:
pd.DataFrame(cats, columns=['cat', 'id', 'desc']).to_json('/data/datasets/imagenet2012/cat_ids_desc.json')

In [45]:
pd.read_json('/data/datasets/imagenet2012/cat_ids_desc.json')['id'].values

array(['n01440764', 'n01443537', 'n01484850', ..., 'n13133613',
       'n15075141', 'brad'], dtype=object)